# Calculations of supercurrents in a proximitized semiconducting 3D nanowire
The simulations in this notebook generate data that is used for the plotting of the theoretical figures of the paper.

Some simulations are expensive and took ~5hrs on 300 cores. Therefore we often cut up the simulation in parts using the `funcs.chunks` function. The resulting (partial) data is saved in `pandas.DataFrame`s.

To view the resulting data (after joining the parts with `pandas.concat`) see [paper-figures.ipynb](paper-figures.ipynb) for the figures from the paper or [explore-data.ipynb](explore-data.ipynb) to interactively explore the data.

# The Python conda environment that is used
* To install, save the following code as `environment.yml`, install `conda` and run: `conda env create -f environment.yml`
```yaml
name: python3
channels:
  - conda-forge
dependencies:
  - python=3.5
  - anaconda=4.3
  - kwant=1.2.2
  - discretizer=0.4.3
  - holoviews=1.7.0
  - xarray=0.9.1
  - hpc05  # This is only needed if you use a PBS cluster with headnode
```

## Notes
* Make sure that a folder `tmp` and `data` exists when running the simulations.

In [ ]:
try:
    # If this can be imported, it assumes you have access to a PBS cluster
    # with a headnode where you started a cluster
    import hpc05
    client = hpc05.Client(profile='pbs', timeout=60)
    print("Connected to hpc05")
except ImportError:
    # Otherwise start an ipyparallel ipcluster locally
    from ipyparallel import Client
    client = Client()
    print("Connected to local engines")

In [ ]:
dview = client[:]
dview.use_dill()
lview = client.load_balanced_view()
print('Connected to {} engines.'.format(len(dview)))

In [ ]:
# Make sure this folder is in your $PYTHONPATH
# or set the correct location in the next line (run after all engines are connected)
%px import sys, os; sys.path.append(os.path.expanduser('~/Work/nanowire_current/'))

In [ ]:
# Standard library imports
from functools import partial
from itertools import product, repeat
import os.path
import types

# Related third party imports
import holoviews as hv
import kwant
import numpy as np
import pandas as pd
import sympy.interactive

# Local imports
import funcs
from funcs import constants

%matplotlib inline
hv.notebook_extension()
sympy.interactive.init_printing('mathjax')

# Creating some wires as example

In [ ]:
# Create system with infinite leads
syst_pars = dict(a=8, A_in_SC=True, angle=0, site_disorder=False, holes=True, L=200, L_sc=80,
                 phi=135, r1=50, r2=70, shape='circle', spin=True, with_leads=True,
                 with_shell=True, with_vlead=True)

syst, hopping = funcs.make_3d_wire(**syst_pars)
kwant.plot(syst)

In [ ]:
%matplotlib notebook
# Create system without leads and bigger SC regions
syst_pars['L_sc'] = 40
syst_pars['L'] = 40
syst, hopping = funcs.make_3d_wire(**syst_pars)
kwant.plot(syst)
plt.show()

In [ ]:
# Create system without leads
syst_pars['with_leads'] = False
syst, hopping = funcs.make_3d_wire(**syst_pars)
kwant.plot(syst)

# Mean free path calculation
See also [mean-free-path.ipynb](mean-free-path.ipynb) for the data processing

In [ ]:
syst_pars = dict(a=8, A_in_SC=True, angle=0, site_disorder=True, holes=False, L_sc=8,
                 phi=135, r1=50, r2=70, shape='circle', spin=True, with_leads=True,
                 with_shell=False, with_vlead=False)

ham_pars = dict(alpha=20, B_x=0, B_y=0, B_z=0, Delta=0, g=50, mu_B=constants.mu_B,
                orbital=True, t=constants.t, V=lambda x: 0)

Ls = np.arange(80, 2000, 80)
salts = np.arange(0, 30)
disorders = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110]
mus = [10, 15, 20, 25, 30, 35, 40, 45, 50]

vals = list(product(salts, disorders, Ls, mus))

def transmission(x, syst_pars=syst_pars, ham_pars=ham_pars):
    import funcs, types, kwant
    syst, hopping = funcs.make_3d_wire(**dict(**syst_pars, L=x[2]))
    p = types.SimpleNamespace(**ham_pars, salt=x[0], disorder=x[1], mu=x[3])
    smatrix = kwant.smatrix(syst, args=[p])
    return smatrix.transmission(0, 1), smatrix.num_propagating(0)

trans = lview.map_async(transmission, vals)
trans.wait_interactive()
trans = trans.result()

df = pd.DataFrame(vals, columns=['salt', 'disorder', 'L', 'mu'])
df = pd.concat((pd.DataFrame(trans, columns=['transmission', 'num_propagating']), df), axis=1)
df = df.assign(**syst_pars).assign(**ham_pars).assign(**constants.__dict__)
df = df.assign(git_hash=funcs.get_git_revision_hash())
df.to_hdf('data/mean_free_path.hdf', 'all_data', mode='w')

# $B_x(I_c)$ no disorder, combinations of effects and geometries:

In [ ]:
syst_pars = dict(a=8, A_in_SC=True, angle=0, site_disorder=False, holes=True, phi=135,
                 r1=50, r2=70, shape='circle', spin=True, 
                 with_shell=True, with_vlead=True)

ham_pars = dict(B_y=0, B_z=0, Delta=60, mu_B=constants.mu_B,
                t=constants.t, t_interface=7/8*constants.t, V=lambda x: 0)

Ts = [0.1, 0.5, 1]
orbital_bools = [False, True]
gs = [0, 50]
alphas = [0, 20]
mus = [10, 20, 30, 40, 50]
Ls = [80, 160, 320, 640]
leads = [(True, 8), (False, 400)]
Bs = np.linspace(0, 2, 100)

vals = list(product(Ts, Ls, orbital_bools, gs, alphas, mus, leads, Bs))

def current(x, syst_pars=syst_pars, ham_pars=ham_pars):
    import kwant, types, funcs
    syst, hopping = funcs.make_3d_wire(**dict(**syst_pars, L=x[1], with_leads=x[6][0], L_sc=x[6][1]))
    p = types.SimpleNamespace(**ham_pars, B_x=x[7], orbital=x[2], g=x[3], alpha=x[4], mu=x[5])
    return funcs.I_c(syst, hopping, p, T=x[0])

for i, chunk in enumerate(funcs.chunks(vals, 3000)):
    fname = 'tmp/I_c_B_x_no_disorder{}.hdf'.format(i)
    if not os.path.exists(fname):
        I = lview.map_async(current, chunk)
        I.wait_interactive()
        current_phase = I.result()
        
        df = pd.DataFrame(chunk, columns=['T', 'L', 'orbital', 'g', 'alpha', 'mu', 'leads', 'B_x'])
        df[['with_leads', 'L_sc']] = df.pop('leads').apply(pd.Series)
        df = pd.concat((pd.DataFrame(current_phase), df), axis=1)
        df = df.assign(**syst_pars).assign(**ham_pars).assign(**constants.__dict__)
        df = df.assign(git_hash=funcs.get_git_revision_hash())
        df.to_hdf(fname, 'all_data', mode='w')

# $B_x(I_c)$ with disorder:
* Main paper Figure 4
* Supplementary materials Figure 8

In [ ]:
syst_pars = dict(a=8, A_in_SC=True, angle=0, site_disorder=True, holes=True, L=640, 
                 L_sc=8, phi=135, r1=50, r2=70, shape='circle', spin=True,
                 with_leads=True, with_shell=True, with_vlead=True)

ham_pars = dict(B_y=0, B_z=0, Delta=60, mu_B=constants.mu_B,
                t=constants.t, t_interface=7/8*constants.t, V=lambda x: 0)

T = 1
Bs = np.linspace(0, 1, 100)

# Uncomment for having all combinations of effects
# salts = [0]
# disorders = [0, 65, 80]
# orbital_bools = [False, True]
# gs = [0, 50]
# alphas = [0, 20]
# mus = [10, 30]
# vals = list(product(salts, disorders, orbital_bools, gs, alphas, mus, Bs))

# These are all the combinations of effects that are showed in Fig. 4 of the main paper.
vals = [[(0, 0, True, 50, 20, 10, B),
         (0, 65, True, 50, 20, 10, B),
         (0, 0, False, 50, 20, 10, B),
         (0, 0, True, 50, 0, 10, B),
         (0, 0, False, 50, 0, 10, B),
         (0, 0, True, 50, 20, 30, B),
         (0, 80, True, 50, 20, 30, B),
         (0, 0, False, 50, 20, 30, B),
         (0, 0, True, 50, 0, 30, B),
         (0, 0, False, 50, 0, 30, B)]
         for B in Bs]
vals = sum(vals, [])

print(len(vals))

def current(x, syst_pars=syst_pars, ham_pars=ham_pars, T=T):
    import kwant, types, funcs
    syst, hopping = funcs.make_3d_wire(**syst_pars)
    p = types.SimpleNamespace(**ham_pars, salt=x[0], disorder=x[1], orbital=x[2], 
                              g=x[3], alpha=x[4], mu=x[5], B_x=x[6])
    return funcs.I_c(syst, hopping, p, T=T, N_brute=31)

for i, chunk in enumerate(funcs.chunks(vals, 5000)):
    fname = 'tmp/I_c_B_x_with_and_without_disorder_L640nm_mu30meV_{}.hdf'.format(i)
    if not os.path.exists(fname):
        I = lview.map_async(current, chunk)
        I.wait_interactive()
        current_phase = I.result()
        
        df = pd.DataFrame(chunk, columns=['salt', 'disorder', 'orbital', 'g', 'alpha', 'mu', 'B_x'])
        df = pd.concat((pd.DataFrame(current_phase), df), axis=1)
        df = df.assign(**syst_pars).assign(**ham_pars).assign(**constants.__dict__)
        df = df.assign(T=T, git_hash=funcs.get_git_revision_hash())
        df.to_hdf(fname, 'all_data', mode='w')

# Rotation of the magnetic field in the $xy$-plane
* Supplementary materials Figure 10

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/3D_Spherical.svg/558px-3D_Spherical.svg.png)

In [ ]:
syst_pars = dict(a=8, A_in_SC=True, angle=0, site_disorder=False, holes=True, L=640, L_sc=8,
                 phi=135, r1=50, r2=70, shape='circle', spin=True, 
                 with_leads=True, with_shell=True, with_vlead=True)

ham_pars = dict(alpha=20, B_z=0, Delta=60, g=50, 
                mu=30, mu_B=constants.mu_B, orbital=True, 
                t=constants.t, t_interface=7/8*constants.t, V=lambda x: 0)

T = 1

Bs = np.linspace(0, 1, 100)
angles = np.arange(0, 195, 15)
thetas = [angle for angle in angles for B in Bs]
B_xs = [B * np.cos(angle * np.pi / 180) for angle in angles for B in Bs]
B_ys = [B * np.sin(angle * np.pi / 180) for angle in angles for B in Bs]
vals = list(zip(B_xs, B_ys))

def fun(x, syst_pars=syst_pars, ham_pars=ham_pars, T=T):
    import kwant, types, funcs
    p = types.SimpleNamespace(**ham_pars, B_x=x[0], B_y=x[1])
    syst, hopping = funcs.make_3d_wire(**syst_pars)
    return funcs.I_c(syst, hopping, p, T)

I = lview.map_async(fun, vals)
I.wait_interactive()
current_phase = I.result()

df = pd.DataFrame(vals, columns=['B_x', 'B_y'])
df['theta'] = thetas
df['B'] = np.sqrt(df['B_x']**2 + df['B_y']**2)
df = pd.concat((pd.DataFrame(current_phase), df), axis=1)
df = df.assign(**syst_pars).assign(**ham_pars).assign(**constants.__dict__)
df = df.assign(T=T).assign(git_hash=funcs.get_git_revision_hash())
df.to_hdf('data/rotation_of_field_xy.hdf', 'all_data', mode='w')

# Tuning the induced gap

In [ ]:
# Create system with infinite leads
syst_pars = dict(a=8, A_in_SC=True, angle=0, site_disorder=False, holes=True,
                 L=80, L_sc=8, phi=135, r1=50, r2=70, shape='circle',
                 spin=True, with_leads=True, with_shell=True, with_vlead=True)

ham_pars = dict(alpha=20, B_x=0, B_y=0, B_z=0, g=50, 
                mu_B=constants.mu_B, orbital=True, 
                t=constants.t, t_interface=7/8*constants.t, V=lambda x: 0)

mus = np.linspace(2, 30, 50)
Deltas = np.linspace(0, 100, 50)

def evs(x, syst_pars=syst_pars, ham_pars=ham_pars):
    import kwant, types, funcs
    import numpy as np
    syst, _ = funcs.make_3d_wire(**syst_pars)
    lead = syst.leads[1]
    p = types.SimpleNamespace(**ham_pars, Delta=x[0], mu=x[1])
    h0 = lead.cell_hamiltonian(args=[p])
    t0 = lead.inter_cell_hopping(args=[p])
    ham = h0 + t0 + t0.conj().T
    ev = np.linalg.eigvalsh(ham)
    return np.abs(ev).min()

vals = list(product(Deltas, mus))
gap = lview.map_async(evs, vals)
gap.wait_interactive()
gap = np.reshape(gap.result(), (len(Deltas), -1)) 
max_gap = max(np.min(gap, axis=1))
print("Max gap is {} meV".format(max_gap))

In [ ]:
hv.Curve((Deltas, gap.min(axis=1)), kdims=['$\Delta$'], vdims=['$E_{gap}$'], label="$E_{gap}$ over range of $\Delta$'s")

In [ ]:
Delta_ind = {delta: hv.Path((mus, evs), kdims=['$\mu$', ('E_gap', '$E_{gap}$')]) 
      for evs, delta in zip(gap, Deltas)}

hm = hv.HoloMap(Delta_ind, kdims=['$\Delta$'])
(hm.select(E_gap=(0.2, 0.40)) * hv.HLine(0.25))

# $I_c(B_x, V)$

In [ ]:
# Plot test
syst_pars = dict(a=8, A_in_SC=True, angle=0, site_disorder=False, holes=True,
                   L=640, L_sc=8, phi=135, r1=50, r2=70, shape='circle',
                   spin=True, with_leads=True, with_shell=True, with_vlead=True)

syst, hopping = funcs.make_3d_wire(**syst_pars)
gate_fun = funcs.gate(syst, 100, 160)
sites = [gate_fun(pos[0]) for pos in [i.pos for i in syst.sites]]
kwant.plot(syst, site_lw=0, site_color=sites, colorbar=False)

## Gate simulation with and without disorder
* Main paper figure 5(b)
* Supplementary materials Figure 9

In [ ]:
syst_pars = dict(a=8, A_in_SC=True, angle=0, site_disorder=True, holes=True,
                 L=640, L_sc=8, phi=135, r1=50, r2=70, shape='circle',
                 spin=True, with_leads=True, with_shell=True, with_vlead=True)

ham_pars = dict(alpha=20, B_y=0, B_z=0, Delta=60, g=50, mu=30,
                mu_B=constants.mu_B, orbital=True, t=constants.t,
                t_interface=7/8*constants.t, salt=0)

T = 1
Bs = np.linspace(0, 0.5, 100)
Vs = np.linspace(0, 7.5, 100)
gate_sizes = [640]
disorders = [80, 0]

def current(x, syst_pars=syst_pars, ham_pars=ham_pars, T=T):
    import kwant, types, funcs
    syst, hopping = funcs.make_3d_wire(**syst_pars)
    p = types.SimpleNamespace(**ham_pars, B_x=x[3],
                              V=funcs.gate(syst, V=x[2], gate_size=x[0]),
                              disorder=x[1])
    return funcs.I_c(syst, hopping, p, T)

vals = list(product(gate_sizes, disorders, Vs, Bs))

for i, chunk in enumerate(funcs.chunks(vals, 10000)):
    fname = "tmp/current_as_function_of_gate_and_B_x_with_and_without_disorder_mu30meV_zoomin{}.h5".format(i)
    if not os.path.exists(fname):
        I = lview.map_async(current, chunk)
        I.wait_interactive()
        current_phase = I.result()

        df = pd.DataFrame(chunk, columns=['gate_size', 'disorder', 'V', 'B_x'])
        df = pd.concat((pd.DataFrame(current_phase), df), axis=1)
        df = df.assign(**syst_pars).assign(**ham_pars).assign(**constants.__dict__)
        df = df.assign(T=T).assign(git_hash=funcs.get_git_revision_hash())
        df.to_hdf(fname, 'all_data', mode='w')